# Onderzoek naar gemeentelijke financieën

De bewoners van Nijkerk geven aan dat zij in de wijk een aantal problemen ondervinden, namelijk:
- Veel leegstand;
- Hogere WOZ-waarde dan normaal;
- Onverstanindige uitgaven door de gemeente.

Aan de hand van data over gemeentelijke financieën van alle gemeenten in Nederland willen wij kijken of Nijkerk op deze aspecten werkelijk afwijkt van het gemiddelde. Eerst laden we de tabellen in.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving gemeente_begroting20-24.xlsx to gemeente_begroting20-24.xlsx
Saving gemeente_jaarrekening20-22.xlsx to gemeente_jaarrekening20-22.xlsx


We laden de bestanden in als Pandas DataFrame en laten even de `head` zien van de jaarrekening voor een overzicht van de datastructuur (de tabel over de begroting heeft dezelfde structuur, maar dan met meer jaartallen).

In [2]:
import pandas as pd

begroting = pd.read_excel("gemeente_begroting20-24.xlsx")
jaarrekening = pd.read_excel("gemeente_jaarrekening20-22.xlsx")

jaarrekening.head()

,Gemeenten,Baten per inwoner per taakveld|2020,Baten per inwoner per taakveld|2021,Baten per inwoner per taakveld|2022,Lasten per inwoner per taakveld|2020,Lasten per inwoner per taakveld|2021,Lasten per inwoner per taakveld|2022,Saldo per inwoner per taakveld|2020,Saldo per inwoner per taakveld|2021,Saldo per inwoner per taakveld|2022,...,Saldo per taakveld|2022,Netto schuldquote (correctie)|2020,Netto schuldquote (correctie)|2021,Netto schuldquote (correctie)|2022,Grondexploitatie|2020,Grondexploitatie|2021,Grondexploitatie|2022,Solvabiliteit|2020,Solvabiliteit|2021,Solvabiliteit|2022
0,Aa en Hunze,3246,3164,3580.0,3246,3164,3580.0,0,0,0.0,...,-3.0,13,NaN,3.3,10,NaN,2.3,48,NaN,55.3
1,Aalsmeer,2893,3481,3444.0,2893,3481,3443.0,0,0,0.0,...,9.0,19,27,8.0,15,15,7.0,53,54,51.0
2,Aalten,2599,2754,2917.0,2599,2754,2917.0,0,0,0.0,...,-3.0,26.8,26.1,21.7,2,0.4,0.5,31.1,33.3,36.7
3,Achtkarspelen,3499,3729,3670.0,3498,3729,3670.0,0,0,0.0,...,8.0,39.3,39.6,30.0,1,1.1,-0.4,39.3,39.6,36.1
4,Alblasserdam,4079,3887,3701.0,4078,3887,3701.0,0,0,0.0,...,2.0,37,67,57.0,1,-1,0.0,39,36,33.0


## Businessvragen

We hebben de volgende businessvragen opgesteld over de data. Deze willen we visueel beantwoorden aan de hand van een Power BI dashboard:

1. Hoe groot is het verschil tussen het saldo van de begroting/jaarrekening in Nijkerk vergeleken met de rest van de gemeenten?
2. Hoeveel verdient de gemeente Nijkerk aan baten vergeleken met andere gemeenten?
3. Bewoners in Nijkerk vinden dat er opvalled veel leegstand is; hoe vergelijkt de grondexploitatie met de rest van de gemeenten?
4. Hoe vergelijkt de solvabiliteit van de gemeente Nijkerk met de rest van de gemeenten?
5. Zijn er financiële kengetallen die alarmbellen moeten gaan rinkelen bij de gemeente Nijkerk?

De huidige datastructuur maakt het lastig om trends over de jaren te visualiseren. Voor ons onderzoek hebben we ook alle data over andere gemeenten niet per se nodig. We willen in ieder geval het gemiddelde voor Nederlandse gemeenten van alle indicatoren hebben, en de indicatoren van Nijkerk.

In [3]:
jaarrekening.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 28 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Gemeenten                             342 non-null    object 
 1   Baten per inwoner per taakveld|2020   342 non-null    int64  
 2   Baten per inwoner per taakveld|2021   342 non-null    int64  
 3   Baten per inwoner per taakveld|2022   339 non-null    float64
 4   Lasten per inwoner per taakveld|2020  342 non-null    int64  
 5   Lasten per inwoner per taakveld|2021  342 non-null    int64  
 6   Lasten per inwoner per taakveld|2022  339 non-null    float64
 7   Saldo per inwoner per taakveld|2020   342 non-null    int64  
 8   Saldo per inwoner per taakveld|2021   342 non-null    int64  
 9   Saldo per inwoner per taakveld|2022   339 non-null    float64
 10  Baten naar taakvelden|2020            342 non-null    int64  
 11  Baten naar taakveld

## Data transformatie

Daarnaast zijn er een aantal kolommen met getallen niet geen float of int datatype hebben. Hierdoor kan het gemiddelde niet berekend worden. Tijdens het bekijken van het Excelbestand bleek ook dat soms `-` gebruikt wordt voor missende waarden. Om de kolommen om te kunnen zetten naar het juiste datatype moeten die eerst vervangen worden met een 0.

In [4]:
jaarrekening = jaarrekening.replace(to_replace=["-", ""], value=0).fillna(0)

# Kolommen die naar int veranderd moeten worden
int_kolommen = jaarrekening.columns[1:19]

# Kolommen die naar float veranderd moeten worden
float_kolommen = jaarrekening.columns[19:]

# Verander de datatypes
jaarrekening[int_kolommen] = jaarrekening[int_kolommen].astype(int)
jaarrekening[float_kolommen] = jaarrekening[float_kolommen].astype(float)

Voor de tabel met de begrotingen geldt hetzelfde, hier zullen we dus ook een aantal datatypes moeten veranderen.

In [5]:
begroting = begroting.replace(to_replace=["-", ""], value=0).fillna(0)

int_kolommen2 = begroting.columns[1:31]
float_kolommen2 = begroting.columns[31:]

begroting[int_kolommen2] = begroting[int_kolommen2].astype(int)
begroting[float_kolommen2] = begroting[float_kolommen2].astype(float)

Nu gaan we de belangrijke data uit de tabellen extraheren. Dit zijn de gemiddelde waarden voor alle kolommen, en de waarden voor de gemeente Nijkerk. We gaan deze data sorteren op basis van het jaar, wat in een aparte kolom terecht komt.

In [6]:
# Bereken het gemiddelde per kolom (behalve de gemeentenamen)
jaarrekening_nl = jaarrekening.iloc[:, 1:].mean().round()
begroting_nl = begroting.iloc[:, 1:].mean().round(1)

# Selecteer alleen de data van Nijkerk
jaarrekening_nijkerk = jaarrekening.loc[jaarrekening["Gemeenten"] == "Nijkerk"].squeeze()[1:]
begroting_nijkerk = begroting.loc[begroting["Gemeenten"] == "Nijkerk"].squeeze()[1:]

# Functie die uit de kolomnaam haalt om welk jaar het gaat
# Alle waarden worden op basis hiervan gesorteerd als dictionary
def sorteer_tabel(series_list, jaren):
  jaren = {jaar: [] for jaar in jaren}

  # Loop door alle kolomnamen en check het jaartal
  for series in series_list:
    for kolomnaam in series.index:
      for jaar in jaren:
        if jaar in kolomnaam:
          jaren[jaar].append(series[kolomnaam])

  return jaren

# Maak dictionaries met de juiste waarden onder elk jaartal
jaarrekening_dict = sorteer_tabel([jaarrekening_nl, jaarrekening_nijkerk], ["2020", "2021", "2022"])
begroting_dict = sorteer_tabel([begroting_nl, begroting_nijkerk], ["2020", "2021", "2022", "2023", "2024"])

We maken de nieuwe tabellen met de gemiddelde waarden voor Nederlandse gemeenten en de waarden van Nijkerk voor elke financiële indicator per jaar. We veranderen ook de kolomnamen zodat deze wat duidelijker zijn. Het resultaat van de begroting wordt in de `head` laten zien.

In [7]:
import numpy as np

kolommen = ["Jaar", "Baten taakvelden per inwoner (NL)", "Lasten taakvelden per inwoner (NL)", "Saldo taakvelden per inwoner (NL)",
           "Baten taakvelden per €1.000 (NL)", "Lasten taakvelden per €1.000 (NL)", "Saldo taakvelden per €1.000 (NL)",
           "Netto schuldquote (NL)", "Grondexploitatie (NL)", "Solvabiliteit (NL)", "Baten taakvelden per inwoner (Nijkerk)",
           "Lasten taakvelden per inwoner (Nijkerk)", "Saldo taakvelden per inwoner (Nijkerk)", "Baten taakvelden per €1.000 (Nijkerk)",
           "Lasten taakvelden per €1.000 (Nijkerk)", "Saldo taakvelden per €1.000 (Nijkerk)", "Netto schuldquote (Nijkerk)",
           "Grondexploitatie (Nijkerk)", "Solvabiliteit (Nijkerk)"]

# Creeer de nieuwe DataFrames
jaarrekening_subset = pd.DataFrame(np.array([["2020"] + jaarrekening_dict["2020"],
                                             ["2021"] + jaarrekening_dict["2021"],
                                             ["2022"] + jaarrekening_dict["2022"]]),
                                   columns=kolommen)

begroting_subset = pd.DataFrame(np.array([["2020"] + begroting_dict["2020"],
                                          ["2021"] + begroting_dict["2021"],
                                          ["2022"] + begroting_dict["2022"],
                                          ["2023"] + begroting_dict["2023"],
                                          ["2024"] + begroting_dict["2024"]]),
                                columns=kolommen)

begroting_subset.head()

,Jaar,Baten taakvelden per inwoner (NL),Lasten taakvelden per inwoner (NL),Saldo taakvelden per inwoner (NL),Baten taakvelden per €1.000 (NL),Lasten taakvelden per €1.000 (NL),Saldo taakvelden per €1.000 (NL),Netto schuldquote (NL),Grondexploitatie (NL),Solvabiliteit (NL),Baten taakvelden per inwoner (Nijkerk),Lasten taakvelden per inwoner (Nijkerk),Saldo taakvelden per inwoner (Nijkerk),Baten taakvelden per €1.000 (Nijkerk),Lasten taakvelden per €1.000 (Nijkerk),Saldo taakvelden per €1.000 (Nijkerk),Netto schuldquote (Nijkerk),Grondexploitatie (Nijkerk),Solvabiliteit (Nijkerk)
0,2020,2969.9,2969.9,-0.0,183940.2,183940.5,-0.3,60.1,9.0,29.4,2332,2332,0,100681,100680,1,24.0,29.0,41.0
1,2021,3021.7,3022.0,-0.4,188404.4,188413.4,-9.0,59.1,7.8,31.0,2436,2435,0,106189,106185,4,63.0,24.0,40.0
2,2022,3116.8,3116.8,0.0,193131.1,193131.2,-0.1,57.7,7.2,31.9,2610,2610,0,115680,115686,-6,75.0,22.0,32.0
3,2023,3357.7,3356.1,1.6,208725.4,208683.4,42.0,58.4,6.3,32.5,3362,3362,0,149789,149788,1,76.0,26.0,29.0
4,2024,3379.7,3379.7,-0.0,210036.5,210036.4,0.1,61.0,4.8,32.6,3897,3897,0,175101,175099,2,60.0,1.9,32.0


Met deze gegevens kunnen we nu in Power BI beter trends visualiseren. We slaan de getransformeerde datasets op als Excelbestanden. Aangezien we de originele datasets hebben opgeschoond, kunnen die wellicht ook nog gebruikt worden.

In [8]:
# Datasets met de gemiddelde waarden voor NL en de waarden voor Nijkerk
jaarrekening_subset.to_excel("jaarrekening_subset.xlsx", index=False)
begroting_subset.to_excel("begroting_subset.xlsx", index=False)

# Originele datasets
jaarrekening.to_excel("jaarrekening20-22.xlsx", index=False)
begroting.to_excel("begroting20-24.xlsx", index=False)

## Bronvermelding

Findo - Data Financiën Decentrale overheden. https://findo.nl/jive?cat_open=begroting
